In [5]:
import os
import string
from time import time
import random

import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize

#stop_words = stopwords.words('english')
stop_words = set(stopwords.words('english') + [word.strip("\n") for word in open("remove_words.txt", "r")])
lemmatizer = WordNetLemmatizer()


def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def preprocess(sentence):
    return [w for w in sentence.lower().split() if w not in stop_words]

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

if __name__ == "__main__":

    run_relevant = True
    main_start = time() 

    #articles_df = pd.read_csv("all_articles.csv")
    articles_df = pd.read_csv("labelled_1.csv")
    articles_df.fillna("", inplace=True)

    num_runs = 0

    while(num_runs < 2):
        
        count_dict = {}


        if run_relevant == True:

            articles = list(articles_df[articles_df['relevant'] == 1]['text'])

        else:

            articles = list(articles_df[articles_df['relevant'] == 0]['text'])      

        #print(sorted(((v, k) for k, v in count_dict.items()), reverse=True))
        
        for article in articles:
        
        #start = time()
        
            sentences = sent_tokenize(article)
            
            for sentence in sentences:
                words_in_sentence = list(sentence.split(" "))
                for word_ in words_in_sentence:

                    word_ = word_.lower()
                    word_ = word_.strip()
                    word_ = word_.replace(" ", "")
                    word_ = word_.replace(",", "")
                    word_ = word_.replace(".", "")
                    word_ = word_.replace(":", "")
                    word_ = word_.replace("/", "")
                    word_ = word_.replace("-", "")
                    word_ = word_.replace("(", "")
                    word_ = word_.replace(")", "")

                    if lemmatizer.lemmatize(word_) != 'ha' and lemmatizer.lemmatize(word_) != 'wa':
                        word_ = lemmatizer.lemmatize(word_)

                    if word_ not in stop_words and word_ not in string.punctuation:
                        if word_ not in count_dict.keys():
                            #embeddings_dict[word_] = model.encode(word_)
                            #embeddings_dict[word_] = ""
                            count_dict[word_] = 1

                        else:
                            count_dict[word_] += 1

            #count[0]+=1
            #print('Cell took %.2f seconds to run.' % (time() - start))

        if run_relevant == True:
            relevant_df = pd.DataFrame(pd.Series(count_dict), columns = ['frequency'])
            relevant_df.reset_index(inplace=True)
            relevant_df = relevant_df.rename(columns={'index': 'word'})
            relevant_df = relevant_df.sort_values(by=['frequency'], ascending=False)
            
            # apply normalization techniques
            apply_scale_column = 'frequency'
            relevant_df['frequency_scaled'] = MinMaxScaler().fit_transform(np.array(irrelevant_df[apply_scale_column]).reshape(-1,1))
              
            #relevant_df.to_csv("relevant_words_count.csv")
            
        else:
            irrelevant_df = pd.DataFrame(pd.Series(count_dict), columns = ['frequency'])
            irrelevant_df.reset_index(inplace=True)
            irrelevant_df = irrelevant_df.rename(columns={'index': 'word'})
            irrelevant_df = irrelevant_df.sort_values(by=['frequency'], ascending=False)
            
            # apply normalization techniques
            apply_scale_column = 'frequency'
            irrelevant_df['frequency_scaled'] = MinMaxScaler().fit_transform(np.array(irrelevant_df[apply_scale_column]).reshape(-1,1))
            
            #irrelevant_df.to_csv("irrelevant_words_count.csv")
        
        num_runs += 1
        run_relevant = False

    num_words_relevant = 70
    num_words_irrelevant = 70

    relevant_words = list(relevant_df.head(num_words_relevant).word)
    irrelevant_words = list(irrelevant_df.head(num_words_irrelevant).word)
    common_words = intersection(relevant_words, irrelevant_words)
    
    unique_relevant = [x for x in relevant_words if x not in common_words]
    unique_irrelevant = [x for x in irrelevant_words if x not in common_words]

    print("COMMON WORDS")
    print(common_words)
    
    print("UNIQUE RELEVANT")
    print(unique_relevant)

    print("UNIQUE IRRELEVANT")
    print(unique_irrelevant)

COMMON WORDS
['retirement', 'tax', 'income', 'plan', 'pension', 'financial', 'saving', 'money', 'rate', 'home', 'time', 'pay', 'credit', 'benefit', 'loan', 'need', 'cost', 'investment', 'mortgage', 'payment', 'child', 'fund', 'interest', 'help', 'cent', 'account', 'work', 'state', 'million', 'month', 'government', 'health', 'company', 'service', 'insurance', 'asset', 'property', 'even', 'change', 'long', 'bank', 'com', 'good', 'return', 'increase', 'want', 'business', 'market']
UNIQUE RELEVANT
['percent', 'family', 'debt', 'student', 'contribution', 'age', 'employee', 'care', 'amount', 'woman', 'worker', 'save', 'life', 'employer', 'bill', 'expense', 'cash', 'school', 'mean', 'free', 'end', 'report']
UNIQUE IRRELEVANT
['price', 'investor', 'stock', 'security', 'share', 'value', 'manager', 'product', 'day', 'risk', 'information', 'sale', 'estate', 'billion', 'member', 'term', 'client', 'use', 'customer', 'high', 'real', 'fee']


In [7]:
pd.set_option('display.max_rows', 1000)
relevant_df.head(70)

,word,frequency
90,retirement,566
100,tax,546
130,income,414
83,plan,402
82,pension,397
91,financial,350
183,saving,341
134,money,322
162,rate,310
8,home,277


In [9]:
irrelevant_df.head(70)

,word,frequency
40,fund,1082
35,investment,772
26,market,680
615,company,621
860,service,610
83,time,563
916,tax,548
110,rate,547
17,retirement,540
491,financial,530


,word,frequency
0,senator,13
1,reverend,5
2,warnock,9
3,introduces,1
4,bill,33
...,...,...
5387,honeymoon,1
5388,pact,1
5389,inspired,1
5390,sea,1


['senator',
 'reverend',
 'warnock',
 'introduces',
 'bill',
 'ensure',
 'georgia',
 'car',
 'buyer',
 'automaker',
 'fully',
 'benefit',
 'cost-cutting',
 'tax',
 'credits;',
 'today',
 'introduced',
 'affordable',
 'electric',
 'vehicle',
 'america',
 'act',
 'would',
 'create',
 'phase-in',
 'period',
 '(ev)',
 'sourcing',
 'manufacturing',
 'requirement']

Index(['pension', 'said', 'retirement', 'year', 'loan', 'mortgage', 'money',
       'rate', 'student', 'pay', 'ha', 'people', 'plan', 'tax', 'debt', 'also',
       'per', 'say', 'financial', 'new'],
      dtype='object')